In [1]:
pip install opencv-python-headless


In [2]:
import cv2

In [5]:
!wget https://pjreddie.com/media/files/yolov3.weights


--2023-11-28 14:54:23--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  43.5MB/s    in 5.9s    

2023-11-28 14:54:29 (40.0 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [6]:
!wget https://github.com/pjreddie/darknet/blob/master/cfg/yolov3.cfg


--2023-11-28 14:54:49--  https://github.com/pjreddie/darknet/blob/master/cfg/yolov3.cfg
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46207 (45K) [text/plain]
Saving to: ‘yolov3.cfg’

yolov3.cfg          100%[===================>]  45.12K  --.-KB/s    in 0.03s   

2023-11-28 14:54:49 (1.56 MB/s) - ‘yolov3.cfg’ saved [46207/46207]



In [7]:
!wget https://github.com/pjreddie/darknet/blob/master/data/coco.names


--2023-11-28 14:55:13--  https://github.com/pjreddie/darknet/blob/master/data/coco.names
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7317 (7.1K) [text/plain]
Saving to: ‘coco.names’

coco.names          100%[===================>]   7.15K  --.-KB/s    in 0s      

2023-11-28 14:55:13 (64.5 MB/s) - ‘coco.names’ saved [7317/7317]



In [11]:
!wget "https://pjreddie.com/media/files/yolov3.weights"

!wget "https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg"

!wget "https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names"

--2023-11-28 14:58:10--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights.1’

yolov3.weights.1    100%[===================>] 236.52M  43.3MB/s    in 6.1s    

2023-11-28 14:58:17 (38.9 MB/s) - ‘yolov3.weights.1’ saved [248007048/248007048]

--2023-11-28 14:58:17--  https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8342 (8.1K) [text/plain]
Saving to: ‘yolov3.cfg.1’

yolov3.cfg.1        100%[===================>]   8.15K  --.-KB/s    in 0s 

In [14]:
net = cv2.dnn.readNetFromDarknet("yolov3-coco/yolov3.cfg", "yolov3-coco/yolov3.weights")
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]


In [14]:

layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# Capture video from your webcam
cap = cv2.VideoCapture(0)  # Use '0' for default webcam

while True:
    ret, frame = cap.read()
    height, width, channels = frame.shape

    # Perform object detection
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Show information on the screen
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Adjust confidence threshold as needed
                # Get object information
                label = classes[class_id]
                color = (0, 255, 0)  # Green color for the bounding box
                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                cv2.putText(frame, label, (x, y + 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

    # Show the video stream
    cv2.imshow("Webcam", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit
        break

cap.release()
cv2.destroyAllWindows()
